# 결정 트리 실습

사용자 행동 인식 데이터 세트

[원본 데이터셋]
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

데이터 세트 정보:

실험은 19-48세 사이의 30명의 지원자 그룹으로 수행되었습니다. 각자 스마트폰(Samsung Galaxy S II)을 허리에 차고 6가지 활동(
1:WALKING, 
2:WALKING_UPSTAIRS, 
3:WALKING_DOWNSTAIRS, 4:SITTING, 5:STANDING, 6:LAYING)을 수행했습니다. 내장된 가속도계와 자이로스코프를 사용하여 50Hz의 일정한 속도로 3축 선형 가속도와 3축 각속도를 캡처했습니다. 실험은 데이터에 수동으로 레이블을 지정하기 위해 비디오로 녹화되었습니다. 얻은 데이터 세트는 훈련 데이터 생성을 위해 70%, 테스트 데이터 생성을 위해 30%가 선택된 두 세트로 무작위로 분할되었습니다.

센서 신호(가속도계 및 자이로스코프)는 노이즈 필터를 적용하여 사전 처리된 다음 2.56초 및 50% 중첩(128 판독/창)의 고정 너비 슬라이딩 창에서 샘플링되었습니다. 중력 및 신체 운동 성분을 갖는 센서 가속도 신호는 Butterworth 저역 통과 필터를 사용하여 신체 가속도와 중력으로 분리되었습니다. 중력은 저주파 성분만 있다고 가정하므로 차단 주파수가 0.3Hz인 필터를 사용했습니다. 각 창에서 시간 및 주파수 영역에서 변수를 계산하여 특징 벡터를 얻었습니다.

이 데이터 세트에 대한 자세한 내용은 README.txt 파일을 확인하십시오.

참가자 중 한 명과 함께 녹화된 6가지 활동의 예를 포함하는 실험 비디오는 다음 링크에서 볼 수 있습니다.

https://www.youtube.com/watch?v=XOEN9W05_4A]


**Mission**

수집된 데이터를 기반으로 결정트리를 이용해 어떤 행동을 하고 있는지 예측해 봅시다.

In [1]:
# ## 코랩을 사용할 때
# #드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

# #현재 작업 위치 이동
# #띄어쓰기에 \붙일 것
# %cd /content/drive/Othercomputers/내\ 노트북_before/Devpy/13.머신러닝

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

## 데이터 로드

1. 피처만 있는 파일을 로드
2. 훈련데이터와 피처를 합쳐 훈련 데이터 프레임 만듬
3. 데스트데이터와 피처를 합쳐 테스트 데이터 프레임 만듬

In [3]:
# 피처이름 로드 
feature_name_df = pd.read_csv('./dataset/human_activity/features_new.txt', header=None,names=['column_name'])


In [4]:
# 피처이름을 리스트로
feature_name = feature_name_df['column_name'].values.tolist()

In [5]:
# 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
X_train = pd.read_csv('./dataset/human_activity/train/X_train.txt',sep='\s+', names=feature_name)
X_test = pd.read_csv('./dataset/human_activity/test/X_test.txt',sep='\s+', names=feature_name)

In [6]:
# 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
from argparse import Action

y_train = pd.read_csv('./dataset/human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
y_test = pd.read_csv('./dataset/human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])

In [20]:
#action 피처의 class별 갯수
y_test.value_counts()

action
6         537
5         532
1         496
4         491
2         471
3         420
dtype: int64

In [21]:
y_train.value_counts()

action
6         1407
5         1374
4         1286
1         1226
2         1073
3          986
dtype: int64

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# DesisionTree분류 클래스 사용(randomstate 156)
dt_clf = DecisionTreeClassifier(random_state=156,min_samples_leaf=16,max_depth=8)

# 훈련
dt_clf.fit(X_train , y_train)

# 예측
pred = dt_clf.predict(X_test)

#평가
accuracy_score(y_test,pred)

0.8605361384458772

In [9]:
# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

DecisionTreeClassifier 기본 하이퍼 파라미터:
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 16, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 156, 'splitter': 'best'}


## 하이퍼파라미터 튜닝1

In [10]:
from sklearn.model_selection import GridSearchCV

#max_depth를 바꿔가며 훈련해보자
#나무의 깊이를 얼마만큼 성장시킬것인가

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24],
    'min_samples_split' : [16,28]
}


In [11]:
from sklearn.model_selection import GridSearchCV

# GridSearchCV 사용
grid_clf = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5)
grid_clf.fit(X_train,y_train)

# 훈련
print('최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_,
                                                  grid_clf.best_score_))


최적 하이퍼 파라미터:{'max_depth': 8, 'min_samples_split': 16}, 최적 평균 정확도:0.862


In [12]:
# GridSearchCV객체의 cv_results_ 속성을 DataFrame으로 생성.
grid_clf_df = pd.DataFrame(grid_clf.cv_results_)

In [13]:
#베스트 분류기 선택
best_clf = grid_clf.best_estimator_


#예측
pred = best_clf.predict(X_test)


#평가
accuracy_score(y_test,pred)

0.8605361384458772

```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

## 하이퍼파라미터 튜닝2

In [14]:
#하이퍼파라미터
# params = {
#     'max_depth' : [ 6 , 8, 16 ,24],
#     'min_samples_split' : [16,28],
# }


In [15]:
# GridSearchCV 사용
# grid_clf = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=3)
# grid_clf.fit(X_train,y_train)

# 훈련
# print('최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_,
#                                                   grid_clf.best_score_))


In [16]:
#베스트 분류기 선택
# best_clf = grid_clf.best_estimator_


#예측
# pred = best_clf.predict(X_test)


#평가
# accuracy_score(y_test,pred)

```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

---

**[생활탐구]**


```
방문을 꼭 닫은채 공부하고 있다는 보검이...
정말 보검이는 공부하고 있을까요?
생체 신호 측정기를 착용한 보검이로 부터 아래와 같은 신호를 가져왔습니다. 
지금 현재 보검이는 무엇을 하고 있나요?

[보검이 생체 신호 파일]X_quiz.txt 

```

In [17]:
#신호 읽어서 예측해보기
X_quiz = pd.read_csv('./dataset/human_activity/X_quiz55.txt',header=None,names=feature_name)
X_quiz

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.173769,-0.024869,-0.094733,-0.893862,-0.944204,-0.960824,-0.904192,-0.950321,-0.967834,-0.923737,...,-0.267475,0.199002,-0.324917,-0.089103,0.10531,0.183368,-0.169434,0.751465,-0.484928,-0.505


In [18]:
# q_dtc = DecisionTreeClassifier(random_state=156,min_samples_leaf=2,max_depth=3)



pred = best_clf.predict(X_quiz)

In [19]:
pred

array([6], dtype=int64)